In [ ]:
import pandas as pd
import numpy as np
import math 
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
subscriptions = pd.read_csv('subscriptionsdataFINAL.csv') 

In [ ]:
visitation_part1 = pd.read_csv('visitationdataPART1.csv') 

visitation_part2 = pd.read_csv('visitationdataPART2.csv') 

In [ ]:
subscriptions.head()

In [ ]:
visitation_part1.head()

In [ ]:
visitation_part2.head()

In [ ]:
#not needed
#from dateutil import parser, tz
#LDN=tz.gettz("Europe/London")
#subscriptions['Join Date'] = subscriptions['Join Date'].apply(lambda x: parser.parse(x).replace(tzinfo=LDN))

In [ ]:
#convert the join and end date to_date
subscriptions['Join Date'] =  pd.to_datetime(subscriptions['Join Date'].str.upper(), format='%b-%y', yearfirst=False)
subscriptions['End Date'] =  pd.to_datetime(subscriptions['End Date'].str.upper(), format='%b-%y', yearfirst=False)
subscriptions.head()

In [ ]:
subscriptions.dtypes

In [ ]:
subscriptions.describe()

In [ ]:
date_min = min (subscriptions['Join Date'].min(),subscriptions['End Date'].min() )
date_max = max (subscriptions['Join Date'].max(),subscriptions['End Date'].max() )
date_min, date_max

In [ ]:
dates = pd.date_range(date_min, date_max, freq = 'MS')
dates

In [ ]:
##we need to know each month kow many activemembers we have to calculate the churn 
##create the dataframe to help us calculate it
customers_total = pd.DataFrame({'Date':dates})
customers_total.head(), customers_total.shape

In [ ]:
##add total month membership in dataframe subscriptions
subscriptions['Duration'] = np.where(subscriptions['End Date'].isna(), 
                                     (date_max - subscriptions['Join Date']), 
                                     (subscriptions['End Date'] - subscriptions['Join Date']) )
##convert the difference in months
subscriptions['Duration'] = (subscriptions['Duration'] /  np.timedelta64(1, 'M') ).apply(math.floor)
subscriptions.head()

In [ ]:
index_date = (dates == subscriptions.iloc[3,1] ).argmax()
dates[index_date]

In [ ]:
##create list with count of members in each month
members_count =  [0] * len(dates)

In [ ]:
def add_count(join_date,duration, dates, members_count):
    index = (dates == join_date).argmax()
    
    for i in range(index,index + duration+1):
        members_count[i] = members_count[i] +1 
    
    return members_count

In [ ]:
subscriptions.apply(lambda row: add_count(row['Join Date'], row['Duration'],
                                          dates, members_count), axis=1)

In [ ]:
#members_count

In [ ]:
##test of lamda for short dataframe
#subs_short = subscriptions.sort_values(by=['Join Date'])
#limit = 50
#subs_short = subs_short[:limit]
#subs_short = subs_short.sort_values(by=['End Date'])
#subs_short

In [ ]:
##test cont
#members_count_short =  [0] * len(dates)
#subs_short.apply(lambda row: add_count(row['Join Date'], row['Duration'],
#                                         dates, members_count_short), axis=1)
#members_count_short

In [ ]:
customers_total = customers_total.assign(Members= members_count)
customers_total

In [ ]:
## members join 
join_frame = subscriptions.groupby(['Join Date']).count()['CustomerID'].reset_index()
join_frame = join_frame.rename(columns={'Join Date':'Date','CustomerID':'New Members' })
customers_total = pd.merge(customers_total, join_frame, on = 'Date', how='outer')
customers_total

In [ ]:
## members leave 
end_frame = subscriptions.groupby(['End Date']).count()['CustomerID'].reset_index()
end_frame = end_frame.rename(columns={'End Date':'Date','CustomerID':'Cancellations' })
customers_total = pd.merge(customers_total, end_frame, on = 'Date', how='outer')
customers_total

In [ ]:
##calulate churn =  number of ending subscription / total members
customers_total['Churn']= customers_total['Cancellations'] / customers_total['Members']
customers_total

In [ ]:
##calulate membership growth =  number of new members subscription / total members
customers_total['Membership Growth']= customers_total['New Members'] / (customers_total['Members'] - customers_total['New Members'])
customers_total['Membership Growth'] =  customers_total['Membership Growth'].replace([np.inf, -np.inf], np.nan) 
customers_total

In [ ]:
#Churn rate vs Time Graph
plt.figure(figsize=(10,10))
plt.plot(customers_total['Date'], customers_total['Churn'])
plt.title('Churn rate Over Time')
plt.ylabel('Churn rate')
plt.xlabel('Date')
plt.axvline(x=datetime.strptime('2016-01-01', '%Y-%m-%d'), linestyle='--', color = 'r')
plt.show()

In [ ]:
#Membership Growth vs Time Graph
plt.figure(figsize=(10,10))
plt.plot(customers_total['Date'], customers_total['Membership Growth'])
plt.title('Membership Growth Over Time')
plt.ylabel('Membership Growth')
plt.xlabel('Date')
plt.axvline(x=datetime.strptime('2016-01-01', '%Y-%m-%d'), linestyle='--', color = 'r')
plt.show()

In [ ]:
#Membership vs Time Graph
plt.figure(figsize=(10,10))
plt.plot(customers_total['Date'], customers_total['Members'])
plt.title('Members Number Over Time')
plt.ylabel('Member')
plt.xlabel('Date')
#plt.axvline(x=datetime.strptime('2016-01-01', '%Y-%m-%d'), linestyle='--', color = 'r')
plt.show()

In [ ]:
#Changes in number of gyms
clubs = subscriptions.Club.unique()
members_count_club = {club:([0]*len(dates))  for club in clubs}
subscriptions_perclub = subscriptions.groupby(subscriptions.Club)
for club in clubs:
    subscriptions_perclub.get_group(club).apply(lambda row : add_count(row['Join Date'], row['Duration'],
                                                    dates, members_count_club[club]), axis=1 )

members_count_club

In [ ]:
members_per_club = pd.DataFrame({'Date':dates,'Club':clubs[0], 'Members':members_count_club[clubs[0]]})

for club in clubs[1:]:
    club_list = [club]*len(dates);
    club_df = pd.DataFrame({'Date':dates,'Club':club_list, 'Members':members_count_club[club]})
    members_per_club = pd.merge(members_per_club, club_df, on = ['Date','Club','Members'], how='outer')

members_per_club['Members'] = members_per_club['Members'].replace(0,np.nan)
members_per_club.to_csv(r'members_per_club.csv')

In [ ]:
members_per_club

In [ ]:
## members join per club
join_frame_club = subscriptions.groupby(['Join Date', 'Club']).count()['CustomerID'].reset_index()
join_frame_club = join_frame_club.rename(columns={'Join Date':'Date','CustomerID':'New Members' })
members_per_club = pd.merge(members_per_club, join_frame_club, on = ['Date', 'Club'], how='outer')

members_per_club


In [ ]:
## members leave per club
cancel_frame_club = subscriptions.groupby(['End Date', 'Club']).count()['CustomerID'].reset_index()
cancel_frame_club = cancel_frame_club.rename(columns={'End Date':'Date','CustomerID':'Cancellations' })
members_per_club = pd.merge(members_per_club, cancel_frame_club, on = ['Date', 'Club'], how='outer')

members_per_club


In [ ]:
##calulate membership growth =  number of new members subscription / total members
members_per_club['Membership Growth']= members_per_club['New Members'] / (members_per_club['Members'] - members_per_club['New Members'])
members_per_club['Membership Growth'] = members_per_club['Membership Growth'].replace([np.inf, -np.inf], np.nan) 
customers_total

In [ ]:
##calulate churn =  number of ending subscription / total members
members_per_club['Churn']= members_per_club['Cancellations'] / members_per_club['Members']
members_per_club

In [ ]:
for interv in range(4, len(clubs)+1, 4):
    plt.figure(figsize=(15,15))
    for club in clubs[interv-4:interv]:
        churn_club = members_per_club[members_per_club.Club == club]
        plt.plot(churn_club["Date"], churn_club["Churn"], label = club)
        print(club)
    plt.legend()



In [ ]:
data_increase_dates = members_per_club[ ( members_per_club.Date > 
                                       pd.to_datetime(('Sep-15').upper(), format='%b-%y', yearfirst=False) ) & 
                                     ( members_per_club.Date <
                                       pd.to_datetime(('Feb-17').upper(), format='%b-%y', yearfirst=False) ) ]

mean_churn_per_club = data_increase_dates.groupby('Club').Churn.agg(['mean'])

mean_churn_per_club.sort_values(by='mean',ascending=False).plot.bar(figsize=(15,15))

